In [21]:
import pandas as pd
import pickle
import numpy as np
pd.set_option('display.max_columns', 300)

## Step 1: Read in hold out data, scalers, and best model

In [13]:
holdout = pd.read_csv('kc_house_data_test_features.csv', index_col=0)
holdout = holdout.drop(columns=['id'])

In [15]:
holdout.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [16]:
holdout['date'] = holdout['date'].str.slice(0, 8, 1)
holdout['year_sold'] = holdout['date'].str.slice(0, 4, 1)
holdout['month_sold'] = holdout['date'].str.slice(4, 6, 1)
holdout['day_sold'] = holdout['date'].str.slice(6, 8, 1)

In [17]:
holdout['year_sold'] = holdout['year_sold'].astype('int64')
holdout['month_sold'] = holdout['month_sold'].astype('int64')
holdout['day_sold'] = holdout['day_sold'].astype('int64')
holdout = holdout.drop(columns='date')

In [18]:
holdout['yr_renovated'] = holdout['yr_renovated'].replace(0, holdout['yr_built'])

In [19]:
holdout['years_since_update'] = (2020 - holdout['yr_renovated'])

In [22]:
holdout['basement'] = np.where(holdout['sqft_basement']==0, 0, 1)

In [23]:
holdout['bedrooms'] = holdout['bedrooms'].replace(0, holdout['bedrooms'].median())

In [24]:
holdout['q1'] = np.where((holdout['month_sold']==1) | (holdout['month_sold']==2) | (holdout['month_sold']==3), 1, 0)

In [25]:
holdout['q2'] = np.where((holdout['month_sold']==4) | (holdout['month_sold']==5) | (holdout['month_sold']==6), 1, 0)

In [27]:
holdout['q3'] = np.where((holdout['month_sold']==7) | (holdout['month_sold']==8) | (holdout['month_sold']==9), 1, 0)

In [28]:
# import py file
import scraper1

# call function that scrapes school data
school_coordinates = scraper1.top_schools()
school_coordinates.head()

,name,lat,long,ad
0,Cascadia Elementary School,47.695213,-122.337156,"1700 N 90th Street Seattle, WA 98115"
1,Decatur Elementary School,47.685413,-122.283672,"7711 43rd Ave Ne Seattle, WA 98115"
2,Stella Schola,47.671574,-122.160037,"13505 Ne 75th Street Redmond, WA 98052"
3,Tesla Stem High School,47.645720,-122.035450,"4301 228th Ave Ne Redmond, WA 98053"
4,International Community School,47.664105,-122.191210,"11133 Ne 65th St Kirkland, WA 98033"


In [29]:
# house distance to a top school
from geopy.distance import geodesic

def get_distance(lat, long):
    miles = []
    for i in range(school_coordinates.shape[0]):
        school = (school_coordinates.loc[i]['lat'], school_coordinates.loc[i]['long'])
        miles.append(geodesic((lat,long), school).miles)
    return min(miles)

In [30]:
for i in range(holdout.shape[0]):
    lat = holdout.loc[i]['lat']
    long = holdout.loc[i]['long']
    holdout.at[i, 'top_school_distance'] = get_distance(lat, long)

In [31]:
holdout['far_school'] = np.where(holdout['top_school_distance'] > holdout['top_school_distance'].quantile(q=.75), 1, 0)


In [71]:
holdout.shape

(4323, 29)

In [72]:
holdout['grade_03'] = 0

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year_sold,month_sold,day_sold,years_since_update,basement,q1,q2,q3,top_school_distance,far_school,grade_03
0,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,1967,98034,47.7089,-122.241,2020,10918,2014,8,27,53,1,0,0,1,0.701625,0,0
1,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,1967,98034,47.7089,-122.241,2020,10918,2015,2,18,53,1,1,0,0,0.701625,0,0
2,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,2005,98029,47.5472,-121.998,1470,1576,2014,11,7,15,1,0,0,0,2.231332,0,0
3,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,1976,98077,47.7427,-122.071,1160,10565,2014,12,3,44,1,0,0,0,0.962112,0,0
4,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,2005,98059,47.4863,-122.140,2830,7916,2015,1,15,15,0,1,0,0,5.226475,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,2009,98103,47.6993,-122.346,1530,1509,2014,5,21,11,0,0,1,0,0.499877,0,0
4319,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,2014,98146,47.5107,-122.362,1830,7200,2015,2,23,6,0,1,0,0,2.748155,0,0
4320,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,2009,98144,47.5944,-122.299,1020,2007,2014,6,23,11,0,0,1,0,3.436900,0,0
4321,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,2004,98027,47.5345,-122.069,1410,1287,2015,1,16,16,0,1,0,0,2.062624,0,0


In [81]:
# holdout.drop('grade_03', axis=1, inplace=True)

In [84]:
selected_columns = ['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'sqft_living15',
 'sqft_lot15',
 'years_since_update',
 'basement',
 'q2',
 'top_school_distance',
 'far_school']

len(selected_columns)

22

In [75]:
list(holdout.columns)

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'year_sold',
 'month_sold',
 'day_sold',
 'years_since_update',
 'basement',
 'q1',
 'q2',
 'q3',
 'top_school_distance',
 'far_school']

In [87]:
holdout = holdout[selected_columns]

In [88]:
import joblib

In [89]:
scaler = joblib.load('scaler1.pkl') 

In [90]:
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [91]:
holdout.shape

(4323, 22)

In [92]:
scaled_holdout = scaler.transform(holdout)

ValueError: operands could not be broadcast together with shapes (4323,22) (17290,) (4323,22) 

In [101]:
joblib_file = "model1.pkl"  
final_model = joblib.load(joblib_file)
output = final_model.predict(holdout)

In [102]:
output

array([[663725.38930323],
       [663725.38930323],
       [411479.45463271],
       ...,
       [158427.4845367 ],
       [451524.60967036],
       [149306.97073099]])

In [79]:
# Ypredict = final_model.predict(holdout)

In [39]:
# from joblib import load
# scaler = load('scaler.pickle')
# transformed_data = scaler.transform(holdout)

In [99]:
# loaded_model = pickle.load(open('model.pickle', 'rb'))
# result = loaded_model.score(holdout, target)
# print(result)

In [98]:
# #import the pickle module and load method
# from pickle import load
# #deserialize (unpickle) the model to use on a new data set
# rfc=load(open('model.pickle','rb'))
# rfc.predict(holdout)

In [100]:
# final_scaler = read_pickle(scaler.pickle)
# final_model = read_pickle(model.pickle)

## Step 2: Feature Engineering for holdout set
Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.



In [4]:
# transformed_holdout = final_scaler(holdout)

## Step 3: Predict the holdout set

In [5]:
# final_answers = final_model.predict(transformed_holdout)

## Step 4: Export your predictions

In [103]:
output.to_csv('housing_preds_amelia_dahm.csv')

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [104]:
np.savetxt("housing_preds_amelia_dahm.csv", output, delimiter=",")